### Parâmetros iniciais

In [0]:
%pip install openpyxl
import os, re, datetime
import pandas as pd
from pyspark.sql import functions as F, types as T
from io import BytesIO

# Caminho base onde os XLSX estão salvos)
LANDING_BASE = "/Volumes/sinesp/source/landing/sinesp/municipios"

CATALOG  = "sinesp"
SCHEMA   = "bronze"

# Lista de UF exatamente como estão nas abas do Excel
UFS = ["AC","AL","AP","AM","BA","CE","DF","ES","GO","MA","MT","MS",
       "MG","PA","PB","PR","PE","PI","RJ","RN","RS","RO","RR","SC","SP","SE","TO"]

### Achar o arquivo XLSX mais recente no landing

In [0]:
from io import BytesIO
import pandas as pd
import datetime

# latest_path vem do passo anterior (string completa para o arquivo .xlsx no DBFS)
# Aqui lemos o conteúdo como binário via Spark
file_bytes = (
    spark.read.format("binaryFile")
    .load(latest_path)        # caminho completo encontrado pelo seu código
    .select("content")
    .head()[0]
)

# Abrimos diretamente no pandas via BytesIO
xls = pd.ExcelFile(BytesIO(file_bytes))

# Lista todas as abas
all_sheets = xls.sheet_names

# Filtra apenas as abas que correspondem às UFs desejadas
sheet_ufs = [s for s in all_sheets if s in UFS]
print("Abas detectadas (UFs):", sheet_ufs)

# Timestamp de ingestão
ingestion_ts = datetime.datetime.utcnow()

# Opcional: acumular tudo em um DataFrame único
accum = None

# Loop pelas abas filtradas
for uf in sheet_ufs:
    df = pd.read_excel(xls, sheet_name=uf, dtype=str)  # Lê a aba no pandas
    # aqui segue seu tratamento...


In [0]:
df_files = (spark.read.format("binaryFile")
            .option("recursiveFileLookup", "true")
            .load(LANDING_BASE)
            .filter(F.lower(F.col("path")).endswith(".xlsx"))
            .select("path","modificationTime"))

_latest = df_files.orderBy(F.col("modificationTime").desc()).limit(1).collect()
if not _latest:
    raise FileNotFoundError(f"Nenhum .xlsx encontrado em {LANDING_BASE}")

latest_path = _latest[0]["path"]
print(f"Usando arquivo XLSX: {latest_path}")


### Helpers de normalização

In [0]:
def canon(name: str) -> str:
    """snake_case sem acentos/símbolos"""
    x = name.strip().lower()
    x = (x
         .replace("ã","a").replace("á","a").replace("â","a").replace("à","a")
         .replace("é","e").replace("ê","e")
         .replace("í","i")
         .replace("ó","o").replace("ô","o")
         .replace("ú","u").replace("ü","u")
         .replace("ç","c"))
    x = re.sub(r"[^a-z0-9_]+","_", x)
    return x

def normalize_columns_pdf(pdf: pd.DataFrame) -> pd.DataFrame:
    mapping = {
        "Cód_IBGE": "cod_ibge",
        "Municipio": "municipio",
        "Município": "municipio",
        "Sigla UF": "uf",
        "Região": "regiao",
        "Regiao": "regiao",
        "Mês/Ano": "mes_ano",
        "Mes/Ano": "mes_ano",
        "Vítimas": "vitimas",
        "Vitimas": "vitimas",
    }
    cols = []
    for c in pdf.columns:
        c2 = mapping.get(c, c)
        cols.append(c2)
    pdf.columns = [canon(c) for c in cols]
    return pdf

# map PT/EN (os 3 primeiros chars do mês)
MONTH_MAP = {
    "jan":1,"fev":2,"mar":3,"abr":4,"mai":5,"jun":6,"jul":7,"ago":8,"set":9,"out":10,"nov":11,"dez":12,
    "jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,"jul":7,"aug":8,"sep":9,"oct":10,"nov":11,"dec":12
}

def parse_mes_ano(s: str):
    """'Jan-18' → (2018, 1)   'Fev-2020' → (2020, 2)"""
    if pd.isna(s):
        return (None, None)
    ss = str(s).strip()
    m3 = ss[:3].lower()
    mm = MONTH_MAP.get(m3)
    # ano: últimos 2 ou 4 dígitos
    yy = re.findall(r"(\d{2,4})", ss)
    year = int(yy[-1]) if yy else None
    if year and year < 100:
        year = 2000 + year
    return (year, mm)


### Cria schema Bronze, se não existir

In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")


### Leitura por as abas com pandas

In [0]:
all_sheets = xls.sheet_names
sheet_ufs  = [s for s in all_sheets if s in UFS]
print("Abas detectadas (UFs):", sheet_ufs)

ingestion_ts = datetime.datetime.utcnow()
accum = None  # opcional: para criar uma tabela única depois

for uf in sheet_ufs:
    print(f"Processando UF: {uf} ...")
    pdf = pd.read_excel(xls, sheet_name=uf, dtype=str)  # mantém tudo como string no começo
    if pdf.empty:
        print(f"  [skip] aba {uf} vazia")
        continue

    pdf = normalize_columns_pdf(pdf)

    # Mantém as colunas principais (as demais trataremos no Silver)
    expected = ["cod_ibge","municipio","uf","regiao","mes_ano","vitimas"]
    existing = [c for c in expected if c in pdf.columns]
    pdf = pdf[existing].copy()

    # Força 'uf' a partir do nome da aba
    pdf["uf"] = uf

    # Casts mínimos
    if "cod_ibge" in pdf.columns:
        pdf["cod_ibge"] = pd.to_numeric(pdf["cod_ibge"], errors="coerce").astype("Int64")
    if "vitimas" in pdf.columns:
        pdf["vitimas"] = pd.to_numeric(pdf["vitimas"], errors="coerce").astype("Int64")

    # Deriva year/month/year_month
    if "mes_ano" in pdf.columns:
        ym = pdf["mes_ano"].apply(parse_mes_ano)
        pdf["year"]  = ym.apply(lambda t: t[0]).astype("Int64")
        pdf["month"] = ym.apply(lambda t: t[1]).astype("Int64")
        pdf["year_month"] = pd.to_datetime(
            {"year": pdf["year"], "month": pdf["month"], "day": 1},
            errors="coerce"
        )

    # Metadados de ingestão
    pdf["_ingestion_ts"] = ingestion_ts
    pdf["_source_path"]  = latest_path

    # Para Spark
    df = spark.createDataFrame(pdf.astype(object))

    table_name = f"{CATALOG}.{SCHEMA}.DadosMunicipio{uf}"
    (df.write
       .format("delta")
       .mode("overwrite")                # primeira carga; depois troque para append + dedup
       .option("overwriteSchema","true")
       .saveAsTable(table_name))
    print(f"  [ok] gravado: {table_name} | linhas: {df.count()}")

In [0]:
display(spark.table("sinesp.bronze.DadosMunicipioAC").limit(10))